In [2]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

device = 'cpu'

#Code loosly based on https://datascience.stackexchange.com/questions/56804/sckit-learn-cross-validation-and-model-retrain
#And our exercise 1a

In [3]:
train = pd.read_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\task1b_ql4jfi6af0\train.csv')

X = train.iloc[:,2:]
Y = train.iloc[:,1]
ID = train.iloc[:,0]
pd.set_option("display.precision", 7)
print(X)
print(Y)
print(X.shape)
print(Y.shape)

       x1    x2    x3    x4    x5
0    0.02  0.05 -0.09 -0.43 -0.08
1   -0.13  0.11 -0.08 -0.29 -0.03
2    0.08  0.06 -0.07 -0.41 -0.03
3    0.02 -0.12  0.01 -0.43 -0.02
4   -0.14 -0.12 -0.08 -0.02 -0.08
..    ...   ...   ...   ...   ...
695  0.31 -0.09  0.04 -0.09  0.03
696 -0.26 -0.01  0.02 -0.40  0.05
697 -0.27 -0.22 -0.01 -0.32 -0.05
698  0.19  0.11 -0.05 -0.27 -0.04
699 -0.09 -0.09 -0.06 -0.41 -0.06

[700 rows x 5 columns]
0     -6.8226793
1     -6.3262905
2     -9.3027282
3     -7.3718925
4     -6.0276470
         ...    
695   -6.1473937
696   -6.2148989
697   -6.8413411
698   -6.5233707
699   -5.6766827
Name: y, Length: 700, dtype: float64
(700, 5)
(700,)



Wir verlieren etwas präzision durch das Laden in den DataFrame (durch float precision)

In [4]:
def transform(X):
    l1 = X
    l2 = X*X
    l3 = np.exp(X)
    l4 = np.cos(X)
    const = np.ones((X.shape[0],1))
    return np.concatenate((l1, l2, l3, l4, const), axis=1)

In [5]:
transformer = FunctionTransformer(transform, validate=True)

In [6]:
#X_Test = np.array([[1, 2, 3, 4, 5], [1, 1,1, 1,1], [1, 1,1, 1,1], [1, 1,1, 1,1]])
#print(transformer.transform(X_Test))

In [7]:
X_trans = transformer.transform(X)
print(X_trans.shape)

(700, 21)


In [12]:
#X_train = X_trans[:600]
#Y_train = Y[:600]
#X_test = X_trans[600:]
#Y_test = Y[600:]

#print(X_train.shape)
#print(X_test.shape)

(600, 21)
(100, 21)


In [22]:
X_train, X_test, y_train, y_test = train_test_split( X_trans, Y, test_size=0.15, random_state=42)

In [23]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)
#best possible is 1

-0.07228702506821572

In [24]:
reg.score(X_train, y_train)

0.04283046974131188

In [25]:
coefficiants = reg.coef_
print("And coefficiants. ")
print(coefficiants)

And coefficiants. 
[  194.08577461   -81.24168448  -274.53071591   164.45373974
  -230.4406407    511.29718488  -231.80911632  2717.5567307
  -632.25281067   647.80189169  -192.35745167    78.93640786
   271.02940433  -161.07022408   232.93553301   844.98754144
  -399.08820478  5721.45850763 -1441.44410965  1510.67846616
     0.        ]


In [7]:
#param_grid = {'alpha' : np.logspace(-2, 20, num=500)}
#kf = KFold(n_splits=10)
#model = Ridge(alpha=1)

#grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_root_mean_squared_error', return_train_score=True)

In [8]:
#best_model = grid_search.fit(X_trans, Y)
#print("Lowest RMSE:" + str(best_model.best_score_))
#best_alpha = best_model.best_estimator_
#print("With alpha: " + str(best_alpha))

#coefficiants = best_model.best_estimator_.coef_
#print("And coefficiants. ")
#print(coefficiants)

store


In [26]:
dictionary = {'Coefficiants': coefficiants}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1bV2.csv', index=False, header=False)

    Coefficiants
0    194.0857746
1    -81.2416845
2   -274.5307159
3    164.4537397
4   -230.4406407
5    511.2971849
6   -231.8091163
7   2717.5567307
8   -632.2528107
9    647.8018917
10  -192.3574517
11    78.9364079
12   271.0294043
13  -161.0702241
14   232.9355330
15   844.9875414
16  -399.0882048
17  5721.4585076
18 -1441.4441096
19  1510.6784662
20     0.0000000


Do it again with 10fold CV

In [9]:
k = 10
kf = KFold(n_splits=k, random_state=None)
model4 = LinearRegression()
acc_score = []
best_acc = 1000

In [12]:
for train_index , test_index in kf.split(X):
        X_train , X_test = X_trans[train_index,:],X_trans[test_index,:]
        Y_train , Y_test = Y[train_index] , Y[test_index]

        model4.fit(X_train,Y_train)
        pred_values = model4.predict(X_test)

        acc = np.sqrt(mean_squared_error(pred_values , Y_test))
        if acc < best_acc:
            best_acc = acc
            best_model = model4
            
        acc_score.append(acc)

In [13]:
print(best_acc)
print(acc_score)

1.5026533942115932
[1.857847963963212, 1.9427760389196727, 2.0791330910686816, 2.099254836793668, 1.5026533942115932, 2.1750940896136473, 2.1292184675783847, 3.0122824475015193, 2.092656530910476, 2.0183780738385697, 1.857847963963212, 1.9427760389196727, 2.0791330910686816, 2.099254836793668, 1.5026533942115932, 2.1750940896136473, 2.1292184675783847, 3.0122824475015193, 2.092656530910476, 2.0183780738385697]


In [14]:
print(model4.coef_)

[   49.32427461   -92.31677187   -49.08284161   142.55312621
  -232.64052742  -633.19361087  -185.6673051   2263.09257
  -434.0833918   1029.36004814   -48.88294897    89.75674898
    46.40565946  -140.74447839   232.99960548 -1314.97583041
  -291.44265695  4587.46481592 -1014.16419639  2291.77628807
     0.        ]


In [16]:
dictionary = {'Coefficiants': model4.coef_}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1bV2.csv', index=False, header=False)

    Coefficiants
0     49.3242746
1    -92.3167719
2    -49.0828416
3    142.5531262
4   -232.6405274
5   -633.1936109
6   -185.6673051
7   2263.0925700
8   -434.0833918
9   1029.3600481
10   -48.8829490
11    89.7567490
12    46.4056595
13  -140.7444784
14   232.9996055
15 -1314.9758304
16  -291.4426569
17  4587.4648159
18 -1014.1641964
19  2291.7762881
20     0.0000000
